In [101]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [102]:
!pip install newspaper3k

In [103]:
!pip install beautifulsoup4

In [104]:
import datetime as dt
import pandas as pd
import numpy as np
import requests
import os.path

from urllib.parse import urlparse
from newspaper import Article
from bs4 import BeautifulSoup
from urllib.parse import unquote, urlparse
from pathlib import PurePosixPath

# Key Note!

Because the assessment system and psychological test results may ONLY be known by psychologists, we need to assume what **Dreams**, **motivations**, and **user test results** are to make it easier to input as data

In [105]:
class data:
    dream = "teknik" #asumsi jurusan impian
    motivation = "menentukan pilihan jurusan yang tepat" #motivasi user using potenZ
    psikotes = "teknik elektro" #asumsi luaran psikotes

# Key Note!

In this project, i just used 1 input variable, it's working and just to make it simple. And this section, i just used the result of psychological test.

In [106]:
#scraping raw data
PATH = "https://medium.com/tag/" + "-".join(data.psikotes.split()) + "/" +"archive"
print(PATH)

https://medium.com/tag/teknik-elektro/archive


In [107]:
# makes a request to the web page and gets its HTML
html_article = requests.get(PATH).content
soup = BeautifulSoup(html_article, 'html.parser')

titles = [] #link untuk setiap judul dari tagging
title = soup.find_all("div", {"class": "postArticle-readMore"})
for i in title:
    titles.append(i.find("a")['href'])

print(titles)

['https://medium.com/a-byte-of-electrical-engineers-life/listrik-fenomena-magis-yang-menjadi-basis-teknologi-modern-9308a49479a3?source=tag_archive---------0-----------------------', 'https://medium.com/a-byte-of-electrical-engineers-life/elektronik-elektronika-emang-ada-hubungannya-sama-model-bb7391f3c479?source=tag_archive---------1-----------------------', 'https://medium.com/a-byte-of-electrical-engineers-life/dunia-rangkaian-terintegrasi-yang-mengagumkan-a0dedaeb2b7b?source=tag_archive---------2-----------------------', 'https://medium.com/a-byte-of-electrical-engineers-life/sekilas-tentang-publikasi-kehidupan-mahasiswa-teknik-elektro-c85b499f6fa0?source=tag_archive---------3-----------------------', 'https://medium.com/@faishalramadhan0/fenomena-planning-fallacy-terhadap-mahasiswa-sebagai-agen-perubahan-b3d4ea2a6bbc?source=tag_archive---------4-----------------------', 'https://medium.com/a-byte-of-electrical-engineers-life/infinite-be-sebuah-produk-teknologi-tinggi-karya-anak-ba

In [108]:
#scraping
def tagging(url):
    html_tag = requests.get(url).content
    soup2 = BeautifulSoup(html_tag, 'html.parser')

    tags = []
    for k in soup2.find_all('a'):
        tags.append(k.get("href"))

    alltags = []
    for i in tags:
        try:
            indexing = PurePosixPath(unquote(urlparse(i).path)).parts[-2] #nama tagged/tag
            if indexing == "tagged":
                alltags.append(PurePosixPath(unquote(urlparse(i).path)).parts[-1])
            elif indexing == "tag":
                alltags.append(PurePosixPath(unquote(urlparse(i).path)).parts[-1])
        except:
            pass

    return alltags

In [109]:
#get the summary and keyword
#ML scraping (uji coba)
for link in titles:
    med_articles = Article(link, language="id") #medium artikel
    med_articles.download()
    med_articles.parse()
    med_articles.nlp()
    
    article_sum = med_articles.summary
    #To extract summary 
    print("Article's Summary:") 
    print(article_sum) 

    article_keyword = med_articles.keywords
    #To extract keywords 
    print("Article's Keywords:") 
    print(article_keyword)
    
    #To extract tags
    print("Article's tags:")
    print(tagging(link))
    print("\n")

Article's Summary:
Namun, orang-orang tetap mempelajari hal-hal yang mengakibatkan fenomena aliran listrik terjadi dan tentunya konsekuensi dari fenomena ini adalah beragam hal-hal praktis yang kita rasakan, seperti penanak nasi, ponsel pintar, mesin cuci, dan sebagainya.
Kalian sudah bisa menebak dong kalo kita bisa membuat benda bermuatan listrik (selanjutnya disebut bermuatan) ini bergerak, kita bisa menghasilkan aliran listrik yang bisa mentenagai ponsel pintar kita!
Proton bermuatan listrik positif, Neutron bermuatan listrik netral (atau tidak bermuatan), dan Elektron bermuatan listrik negatif.
Namun, jangan salah meskipun ukuran mereka berbeda jauh, dari segi muatan listrik, Elektron ini sama kuatnya dengan Proton loh!
Model Rangkaian Listrik SederhanaAliran listrik terjadi karena medan listrik yang dimiliki baterai pada kedua ujung kutubnya mendorong muatan negatif (elektron) dari kutub negatif baterai bergerak melalui konduktor (kabel) untuk bertemu muatan positif pada kutub po

In [110]:
#summary to input >>> recognize behaviour
#ML scraping
splitting_input = data.psikotes.split() #teknik elektro

articles = [] #final keywords
for links in titles:
    med_articles = Article(links, language="id") #medium artikel
    med_articles.download()
    med_articles.parse()
    med_articles.nlp()

    splitting_output = med_articles.summary.split() #summary of articles
    splitting_output2 = med_articles.keywords #keywords of articles
    splitting_output3 = tagging(link) #tagged in articles

    total_splitting_output = splitting_output + splitting_output2 + splitting_output3
    
    dummy = list(set([]))
    for b in total_splitting_output:
        z = b.lower()
        dummy.append(z)

    articles.append(dummy)

print(articles[0])

['namun,', 'orang-orang', 'tetap', 'mempelajari', 'hal-hal', 'yang', 'mengakibatkan', 'fenomena', 'aliran', 'listrik', 'terjadi', 'dan', 'tentunya', 'konsekuensi', 'dari', 'fenomena', 'ini', 'adalah', 'beragam', 'hal-hal', 'praktis', 'yang', 'kita', 'rasakan,', 'seperti', 'penanak', 'nasi,', 'ponsel', 'pintar,', 'mesin', 'cuci,', 'dan', 'sebagainya.', 'kalian', 'sudah', 'bisa', 'menebak', 'dong', 'kalo', 'kita', 'bisa', 'membuat', 'benda', 'bermuatan', 'listrik', '(selanjutnya', 'disebut', 'bermuatan)', 'ini', 'bergerak,', 'kita', 'bisa', 'menghasilkan', 'aliran', 'listrik', 'yang', 'bisa', 'mentenagai', 'ponsel', 'pintar', 'kita!', 'proton', 'bermuatan', 'listrik', 'positif,', 'neutron', 'bermuatan', 'listrik', 'netral', '(atau', 'tidak', 'bermuatan),', 'dan', 'elektron', 'bermuatan', 'listrik', 'negatif.', 'namun,', 'jangan', 'salah', 'meskipun', 'ukuran', 'mereka', 'berbeda', 'jauh,', 'dari', 'segi', 'muatan', 'listrik,', 'elektron', 'ini', 'sama', 'kuatnya', 'dengan', 'proton', 'lo

# Key Note!
Thi section just calculate matches of user keywords with article keywords. The highest three will be a recommendation an articles for user.

In [111]:
#recommend system
#classification the same thing with the higher value
final_comparassion = {}
for idxi, i in enumerate(splitting_input):
    for idxj, j in enumerate(articles):
        if idxi == 0:
            final_comparassion['Article {}'.format(idxj+1)] = j.count(i)
        else:
            final_comparassion['Article {}'.format(idxj+1)] = final_comparassion['Article {}'.format(idxj+1)] + j.count(i)
        
        
print(final_comparassion)

{'Article 1': 0, 'Article 2': 0, 'Article 3': 0, 'Article 4': 3, 'Article 5': 0, 'Article 6': 0, 'Article 7': 27, 'Article 8': 0, 'Article 9': 0, 'Article 10': 16}


In [112]:
#urutin 3 terbaik
val_dict = list(final_comparassion.values())
val_dict.sort(reverse=True)
print(val_dict)

[27, 16, 3, 0, 0, 0, 0, 0, 0, 0]


In [113]:
#make a sequences of data to suggest
key_list = list(final_comparassion.keys())
val_list = list(final_comparassion.values())

final_seq = []
for i in range(0,3):
    position = val_list.index(val_dict[i])
    final_seq.append(key_list[position].split()[-1])
    
#top 3 recommendation
for final in final_seq:
    print(titles[int(final)-1])
    


https://medium.com/@ovianto05/jurusan-teknik-elektro-lulus-jadi-tukang-service-elektronik-f6350b4fe4bd?source=tag_archive---------6-----------------------
https://medium.com/syifnrl/teknik-elektro-telkom-university-af96332ac0da?source=tag_archive---------9-----------------------
https://medium.com/a-byte-of-electrical-engineers-life/sekilas-tentang-publikasi-kehidupan-mahasiswa-teknik-elektro-c85b499f6fa0?source=tag_archive---------3-----------------------
